# Baseball Statistics Database Building Project (SQL)

In this project the goal is to build and work with a database of baseball statistics. First, a database will be constructed using different .csv files. The resulting database will then be re-designed and normalized.

In [1]:
!rm mlb.db
!del mlb.db

'rm' is not recognized as an internal or external command,
operable program or batch file.
Could Not Find C:\Users\apist\Documents\Projects\DQ_Project_10_Creating_and_Analyzing_an_MLB_Database\mlb.db


In [2]:
# Initial Imports and setting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import sqlite3
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)
db = 'mlb.db'

## First Analysis of the Data

In [3]:
#Loading files, since it apparently takes forever
game_log_1 = pd.read_csv('game_log_1.csv', low_memory = False)
game_log_2 = pd.read_csv('game_log_2.csv', header = None, low_memory = False)
game_log_2.columns = game_log_1.columns
game_log = pd.concat([game_log_1, game_log_2], ignore_index= True)

park_codes = pd.read_csv('park_codes.csv', low_memory = False)
person_codes = pd.read_csv('person_codes.csv', low_memory = False)
team_codes = pd.read_csv('team_codes.csv', low_memory = False)

In [4]:
# First analysis of game_log
game_log.head(2)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,...,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,...,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,...,birdd102,Dave Birdsall,9.0,mcvec101,Cal McVey,2.0,wrigh101,Harry Wright,8.0,goulc101,Charlie Gould,3.0,schah101,Harry Schafer,5.0,conef101,Fred Cone,7.0,spala101,Al Spalding,1.0,watef102,Fred Waterman,5.0,forcd101,Davy Force,6.0,mille105,Everett Mills,3.0,allid101,Doug Allison,2.0,hallg101,George Hall,7.0,leona101,Andy Leonard,4.0,braia102,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y


## game_log.csv

* This file 161 columns and 171907 rows, implying this amount of baseball games being in this dataset
* It includes miriad statistics of the games played
* It also includes external statistics (attendance, where the game was being played) 


In [5]:
!cat game_log_fields.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
park_codes.head(2)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN


## park_codes.csv

* This dataset has 9 columns and 252 rows (implying data on 252 ball parks)
* the dataset has a column park_id. This can probably be used to connect this table to the game_log dataset
* name, alias, city, state, start, end and league are provieded for each ball park
* there is also a column called notes. It is not clear what kind of information this column contains

In [7]:
person_codes.head(2)

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN


## person_codes.csv

* This dataset contains information about the players, managers and coaches in the games in game_log

* it is not clear if only people are mentioned that were at one point a player

* id (of player) can probably be used to connect this dataset to others

* other columns:
    * last (last name)
    * first (first_name)
    * player_debut
    * mgr_debut (debut date of the player as a manager)
    * coach_debut
    * ump_debut (debut date of person as an umpire)
    
* The debut dates that do not apply, because for example a player never worked as a coach are filled with null values

* More than 2000 people are in this dataset. It has 7 colunms

In [8]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


## team_codes.csv

This dataset is based on the data of the different teams. It contains information about 150 teams and has 8 columns:

* team_id: id of the team. Probably a good primary key for this dataset
* league
* start, end: years in between which the team was active. If still active end = 0
* city
* nickname
* franch_id: id of the franchise. Not clear what the difference to tram_id is
* seq: unclear what information this holds



## Different Leagues

In [9]:
team_codes['league'].unique()

array(['UA', 'NL', 'PL', nan, 'AA', 'AL', 'FL'], dtype=object)

* UA: United association. Lasted only one season
* AL: American League: The better one of the two professional leagues. Designated hitters allowed (hitters that hit the ball instead of the pitcher)
* NL: National League: Worse than the AL but the only other professional. No designated hitters
* PL: Player's league. Major league that only lasted until 1890
* nan: Null value
* AA:  Second highest minor league after AAA
* FL: Federal league of baseball clubs last season in 1915

## Defensive Numbering of Positions

In the game log dataset each player has a defensive position attributed to them:

1. Pitcher
2. Catcher
3. First Base
4. Second Base
5. Third Base
6. Short Stop (Betwenn Second and Third Base)
7. Left Field
8. Center Field
9. Right Field

These numbers describe what position the player played during the game

Functions to work with SQL commands:

In [10]:
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        return conn.execute(c)

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql_query(q, conn)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [11]:
# turned to raw after running it once
game_log.to_sql('game_log', sqlite3.connect(db), index = False, if_exists = 'append')
park_codes.to_sql('park_codes', sqlite3.connect(db), index = False, if_exists = 'append')
person_codes.to_sql('person_codes', sqlite3.connect(db), index = False, if_exists = 'append')
team_codes.to_sql('team_codes', sqlite3.connect(db), index = False, if_exists = 'append')


In [12]:
c1 = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT
'''
run_command(c1)

In [13]:
#testing the column to be used as a key
q2 = '''
SELECT date || h_name || number_of_game
FROM game_log
LIMIT 5
'''
run_query(q2)

,date || h_name || number_of_game
0,18710504FW10
1,18710505WS30
2,18710506RC10
3,18710508CH10
4,18710509TRO0


In [14]:
c2 = '''
UPDATE game_log
SET game_id = date||h_name||number_of_game
'''
run_command(c2)

In [15]:
# Verifying functionality
q3 = '''
SELECT game_id
FROM game_log
LIMIT 5'''
run_query(q3)

,game_id
0,18710504FW10
1,18710505WS30
2,18710506RC10
3,18710508CH10
4,18710509TRO0


## Opportunities to Normalize the Data:

* Park names: on the learn screen it was discussed that park names are available in the park_codes dataset and in the game_log dataset. The proposed resolution to this is to use the park data in the game_log dataset. I disagree, since then we would have all the park information repeated for every game that is played in a given park. I think it would be smarter to use the data in park_codes and drop the corresponding columns from the game_log dataset
* park_codes[notes]: This column will most likely not be used in the analysis. It would be safe to drop it
* League: Information about the league is mentioned in game_log, team_codes and park_codes. This is redundant. We should just keep it in the team_codes dataset to declutter the remaining tables
* team_id and franchise_id. These seem to be the same. We should drop one.
* first and last names: these should be combined into one column
* team_codes['seq']: if the seq column is not usefull it should be dropped
* the game_log database carries a lot of columns with very little information pertaining to events in the individual games. We should combine as many of them as possible to cut down on the clutter
* game_log[day_of_week]: This field does not provide any information that is not available through the date field
* game_log[v_league]: This column does not contain any new information since the league is the same as the home team's and is also full of null values
*

* possible primary keys:
    * team_codes[team_id]
    * park_codes[park_id]
    * person_codes[id]
    * for game_log we already made a suitable primary key in the last step

## Designing the database in DB Designer

![](dq_db_pic.png)

## The schema we'll be working with

![](mlb_schema_musterlösung.svg)

# Creating the New Tables:

## Creating Table Person

In [16]:
# Create the person table
c3 = '''
CREATE TABLE
IF NOT EXISTS
person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT)'''

run_command(c3)

#insert values into the table

### Insert values into person

In [17]:
#insert values into perso
c4 = '''
INSERT OR IGNORE INTO person
SELECT
id person_id,
first first_name,
last last_name
FROM person_codes
'''
run_command(c4)

Validation of results

In [18]:
q4 = '''SELECT * FROM person LIMIT 5'''
run_query(q4)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


## Creating 'park' and Inserting Values

In [19]:
c5 = '''
CREATE TABLE
IF NOT EXISTS
park(
park_id TEXT PRIMARY KEY,
name TEXT,
nickname TEXT,
city TEXT,
state TEXT,
notes TEXT)'''
run_command(c5)

c6 = '''
INSERT OR IGNORE
INTO park
SELECT
park_id,
name,
aka nickname,
city,
state,
notes
FROM park_codes
'''
run_command(c6)

Validation

In [20]:
q5 = '''SELECT * FROM park LIMIT 5'''
run_query(q5)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


## Creating 'appearance_type'

In [21]:
#CREATE appearance_type table
c7 = '''
CREATE TABLE
IF NOT EXISTS
appearance_type(
appearance_type_id TEXT PRIMARY KEY,
name TEXT,
category TEXT)
'''
run_command(c7)

### Inserting Values Using Pandas

In [22]:
# import the new table as dataframe
appearance_type = pd.read_csv('appearance_type.csv')
appearance_type.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [23]:
appearance_type.to_sql('appearance_type', sqlite3.connect(db), index = False, if_exists= 'append')

In [24]:
run_query('SELECT * FROM appearance_type')

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


## Creating 'league'

In [25]:
c8 = '''
CREATE TABLE
IF NOT EXISTS
league(
league_id TEXT PRIMARY KEY,
name TEXT)
'''
run_command(c8)

### Inserting Values Into 'league'

* UA: United association. Lasted only one season
* AL: American League: The better one of the two professional leagues. Designated hitters allowed (hitters that hit the ball instead of the pitcher)
* NL: National League: Worse than the AL but the only other professional. No designated hitters
* PL: Player's league. Major league that only lasted until 1890
* nan: Null value
* AA:  Second highest minor league after AAA
* FL: Federal league of baseball clubs last season in 1915

In [26]:
c9 = '''
INSERT OR IGNORE
INTO league
VALUES
    ("AA", "American Association"),
    ("AL", "American League"),
    ("FL", "Federal League"),
    ("NL", "National League"),
    ("PL", "Players League"),
    ("UA", "Union Association")
'''
run_command(c9)

In [27]:
run_query('SELECT * FROM league')

,league_id,name
0,AA,American Association
1,AL,American League
2,FL,Federal League
3,NL,National League
4,PL,Players League
5,UA,Union Association


## Creating 'team'

In [28]:
c10 = '''
CREATE TABLE
IF NOT EXISTS
team(
team_id TEXT PRIMARY KEY,
league_id TEXT,
city TEXT,
nickname TEXT,
franch_id TEXT,
FOREIGN KEY (league_id) REFERENCES league(league_id))
'''
run_command(c10)

Keys from old table:

In [29]:
team_codes.columns

Index(['team_id', 'league', 'start', 'end', 'city', 'nickname', 'franch_id',
       'seq'],
      dtype='object')

Inserting values

In [30]:
c = '''
INSERT OR IGNORE
INTO team
SELECT
team_id,
league league_id,
city,
nickname,
franch_id
FROM team_codes
'''
run_command(c)

Validation

In [31]:
run_query('SELECT * FROM team LIMIT 5')

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


## Creating 'game'

In [32]:
# print(game_log.columns)
print(game_log.columns[game_log.columns.str.contains('for', flags = re.I)])
game_log['attendance'].value_counts(dropna = False)
# game_log['forefeit'].head()

Index(['forefeit'], dtype='object')


NaN        53030
0.0         7544
5000.0       836
10000.0      782
2000.0       684
3000.0       683
15000.0      585
4000.0       580
8000.0       574
12000.0      565
6000.0       494
20000.0      472
1000.0       453
2500.0       452
1500.0       405
7000.0       385
25000.0      368
3500.0       363
18000.0      247
30000.0      243
500.0        207
7500.0       194
4500.0       152
1200.0       149
22000.0      148
35000.0      133
9000.0       122
14000.0      121
28000.0      104
16000.0      100
40000.0       96
800.0         95
17000.0       92
11000.0       88
40930.0       85
13000.0       83
300.0         75
6500.0        73
200.0         69
8500.0        68
32000.0       65
5500.0        64
23000.0       61
600.0         57
27000.0       52
1800.0        51
21000.0       45
45000.0       43
38000.0       42
26000.0       39
24000.0       38
36067.0       35
400.0         34
33000.0       34
31000.0       33
100.0         33
3200.0        32
50000.0       32
700.0         

In [33]:
c = '''
CREATE TABLE
IF NOT EXISTS
game(
game_id TEXT PRIMARY KEY,
date INTEGER,
number_of_game INTEGER,
park_id TEXT,
length_outs NUMERIC,
day INTEGER,
completion TEXT,
forfeit TEXT,
protest TEXT,
attendance INTEGER,
length_minutes INTEGER,
aditional_info TEXT,
acquisition_info TEXT,
FOREIGN KEY (park_id) REFERENCES park(park_id)
)
'''
run_command(c)
# forefeit
 

### Insert Values

In [34]:
#testing out the column_names in game_log
print(game_log.columns)
#print(game_log.columns[game_log.columns.str.contains('for', flags = re.I)])
#game_log['attendance'].value_counts(dropna = False)
game_log['day_night'].head()

Index(['date', 'number_of_game', 'day_of_week', 'v_name', 'v_league',
       'v_game_number', 'h_name', 'h_league', 'h_game_number', 'v_score',
       ...
       'h_player_7_name', 'h_player_7_def_pos', 'h_player_8_id',
       'h_player_8_name', 'h_player_8_def_pos', 'h_player_9_id',
       'h_player_9_name', 'h_player_9_def_pos', 'additional_info',
       'acquisition_info'],
      dtype='object', length=161)


0    D
1    D
2    D
3    D
4    D
Name: day_night, dtype: object

CASE to re-format the day_night column to a boolean like day column

In [35]:
q = '''SELECT day_night,
CASE
    WHEN day_night = "D" THEN 1
    ELSE 0
    END
    AS
    day
FROM game_log
LIMIT 5'''
run_query(q)

,day_night,day
0,D,1
1,D,1
2,D,1
3,D,1
4,D,1


Constructing and testing the query that will later be used for data insertion

In [36]:
q = '''SELECT
game_id,
date,
number_of_game,
park_id,
length_outs,
CASE
    WHEN day_night = "D" THEN 1
    ELSE 0
    END
    AS
    day,
completion,
forefeit forfeit,
protest,
attendance,
length_minutes,
additional_info,
acquisition_info

FROM
game_log
LIMIT 5
'''
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54.0,1,None,None,None,200.0,120.0,None,Y
1,18710505WS30,18710505,0,WAS01,54.0,1,None,None,None,5000.0,145.0,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54.0,1,None,None,None,1000.0,140.0,None,Y
3,18710508CH10,18710508,0,CHI01,54.0,1,None,None,None,5000.0,150.0,None,Y
4,18710509TRO0,18710509,0,TRO01,54.0,1,None,None,None,3250.0,145.0,HTBF,Y


Insertion of Data into 'game'

In [37]:
c = '''
INSERT OR IGNORE
INTO
game
SELECT
game_id,
date,
number_of_game,
park_id,
length_outs,
CASE
    WHEN day_night = "D" THEN 1
    ELSE 0
    END
    AS
    day,
completion,
forefeit forfeit,
protest,
attendance,
length_minutes,
additional_info,
acquisition_info

FROM
game_log
'''
run_command(c)

Validation:

In [38]:
run_query('select * FROM game LIMIT 5')

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,aditional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,18710505WS30,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,18710508CH10,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,18710509TRO0,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


## Create 'team_appearance'

In [39]:
game_log.columns[game_log.columns.str.contains('rbi')]

Index(['v_rbi', 'h_rbi', 'winning_rbi_batter_id',
       'winning_rbi_batter_id_name'],
      dtype='object')

In [40]:
run_query('SELECT v_rbi FROM game_log LIMIT 5')

,v_rbi
0,0.0
1,13.0
2,8.0
3,10.0
4,6.0


In [41]:
c = '''
CREATE TABLE
IF NOT EXISTS
team_appearance(
team_id TEXT,
game_id TEXT,
home BOOLEAN,
league_id TEXT,
score INTEGER,
line_score INTEGER,
at_bats INTEGER,
hits INTEGER,
doubles INTEGER,
triples INTEGER,
homeruns INTEGER,
rbi INTEGER,
sacrifice_hits INTEGER,
sacrifice_flies INTEGER,
hit_by_pitch INTEGER,
walks INTEGER,
intentional_walks INTEGER,
strikeouts INTEGER,
stolen_bases INTEGER,
caught_stealing INTEGER,
grounded_into_double INTEGER,
first_catcher_interference INTEGER,
left_on_base INTEGER,
pitchers_used INTEGER,
individual_earned_runs INTEGER,
team_earned_runs INTEGER,
wild_pitches INTEGER,
balks INTEGER,
putouts INTEGER,
assists INTEGER,
errors INTEGER,
passed_balls INTEGER,
double_plays INTEGER,
triple_plays INTEGER,
PRIMARY KEY (team_id, game_id),
FOREIGN KEY (game_id) REFERENCES game(game_id)
FOREIGN KEY (league_id) REFERENCES league(league_id)
FOREIGN KEY (team_id) REFERENCES team(team_id)
)
'''
run_command(c)

Testing out the select statement for data insertion

In [42]:
q = '''
SELECT sql
FROM sqlite_master
WHERE name = "game_log"
AND type = "table";
'''
run_query(q)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [43]:
q = '''
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    
    FROM game_log

UNION

    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log
LIMIT 5
'''
run_query(q)

,h_name,game_id,home,h_league,h_score,h_line_score,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays
0,ALT,18840417CNU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,18840418CNU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,18840419CNU0,0,UA,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,18840424SLU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,18840426SLU0,0,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [44]:
c = '''
INSERT OR IGNORE
INTO team_appearance
SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    
    FROM game_log

UNION

    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log
'''
run_command(c)

In [45]:
run_query('SELECT * FROM team_appearance ORDER BY league_id LIMIT 5')

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,BL1,18720418WS30,0,None,16,4304410,44,14,4.0,0,0.0,9,0,0,0,1,None,0,NaN,1,0,None,8,1,0,0,1.0,0,21,9,5,0.0,1.0,0
1,BL1,18720420WS40,0,None,21,220342710,57,17,NaN,-1,NaN,13,0,0,0,0,None,0,NaN,-1,0,None,9,1,0,0,NaN,0,27,10,6,NaN,-1.0,0
2,BL1,18720422BL10,1,None,14,211033103,49,18,1.0,0,0.0,9,0,0,0,0,None,1,0.0,0,0,None,8,1,3,0,0.0,0,27,8,8,0.0,2.0,0
3,BL1,18720425BL10,1,None,11,1300230002,55,21,0.0,0,0.0,9,0,0,0,1,None,0,0.0,0,0,None,12,2,6,0,3.0,0,33,16,8,1.0,0.0,0
4,BL1,18720427BL10,1,None,22,53017312,56,17,NaN,-1,2.0,15,0,0,0,0,None,0,NaN,-1,0,None,7,1,1,0,0.0,0,27,8,8,3.0,NaN,0


## Create 'person_appearance'

In [46]:
c = '''
CREATE TABLE
IF NOT EXISTS
person_appearance(
appearance_id INTEGER PRIMARY KEY,
person_id TEXT,
team_id TEXT,
game_id TEXT,
appearance_type_id TEXT,
FOREIGN KEY (person_id) REFERENCES person(person_id),
FOREIGN KEY (team_id) REFERENCES team(team_id),
FOREIGN KEY (game_id) REFERENCES game(game_id),
FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
)
'''
run_command(c)

Finding the names of the columns that need to go into this table

In [47]:
player_columns = game_log.columns[game_log.columns.str.contains(r'id$')][1:]
print(player_columns)

Index(['hp_umpire_id', '1b_umpire_id', '2b_umpire_id', '3b_umpire_id',
       'lf_umpire_id', 'rf_umpire_id', 'v_manager_id', 'h_manager_id',
       'winning_pitcher_id', 'losing_pitcher_id', 'saving_pitcher_id',
       'winning_rbi_batter_id', 'v_starting_pitcher_id',
       'h_starting_pitcher_id', 'v_player_1_id', 'v_player_2_id',
       'v_player_3_id', 'v_player_4_id', 'v_player_5_id', 'v_player_6_id',
       'v_player_7_id', 'v_player_8_id', 'v_player_9_id', 'h_player_1_id',
       'h_player_2_id', 'h_player_3_id', 'h_player_4_id', 'h_player_5_id',
       'h_player_6_id', 'h_player_7_id', 'h_player_8_id', 'h_player_9_id'],
      dtype='object')


In [48]:
player_df = pd.DataFrame(player_columns)
player_df.columns = ['person_id']
player_df['person_id'] = pd.concat([player_df['person_id'], player_df['person_id']], ignore_index = True)

# right name
def h_v(string):
    if string[0:2] == 'v_':
        return 'v_name'
    elif string[0:2] == 'h_':
        return 'h_name'
    elif string[0:7] == 'winning':
        return '''CASE WHEN h_score > v_score THEN h_name ELSE v_name END'''
    elif string[0:6] == 'losing':
        return '''CASE WHEN h_score < v_score THEN h_name ELSE v_name END'''
    elif 'umpire' in string:
        return 'NULL'
    elif 'saving' in string:
        return '''CASE WHEN h_score > v_score THEN h_name ELSE v_name END'''
    else:
        return 'NULL'

def app_ty(string):
    if 'umpire' in string:
        return ('u'+ string[0:2]).upper()
    elif 'manager' in string:
        return 'MM'
    elif 'starting_pitcher_id' in string:
        return 'PSP'
    elif 'pitcher' in string:
        return 'A'+string[0].upper()+'P'
    elif 'batter' in string:
        return 'AWB'
    else: 
        pos_nr = re.findall('player_(\d)_id', string)[0]
        return 'O' + pos_nr
    
    
def def_pos(string):
    pos_nr = re.findall('player_(\d)_id', string)
    if len(pos_nr) == 0:
        return 'NULL'
    else:
        return '"D" || CAST('+ string[0]+'_player_' + str(pos_nr[0]) + '_def_pos AS INT)'


player_df['team_id'] = player_df['person_id'].apply(h_v)
player_df['appearance_type'] = player_df['person_id'].apply(app_ty)
player_df['def_pos'] = player_df['person_id'].apply(def_pos)
'''
for column in player_columns:
    if column[0] = v:
        print('v_name')
        print
    
    
    
    print('['+column+']')'''
print(player_df)

                person_id  \
0            hp_umpire_id   
1            1b_umpire_id   
2            2b_umpire_id   
3            3b_umpire_id   
4            lf_umpire_id   
5            rf_umpire_id   
6            v_manager_id   
7            h_manager_id   
8      winning_pitcher_id   
9       losing_pitcher_id   
10      saving_pitcher_id   
11  winning_rbi_batter_id   
12  v_starting_pitcher_id   
13  h_starting_pitcher_id   
14          v_player_1_id   
15          v_player_2_id   
16          v_player_3_id   
17          v_player_4_id   
18          v_player_5_id   
19          v_player_6_id   
20          v_player_7_id   
21          v_player_8_id   
22          v_player_9_id   
23          h_player_1_id   
24          h_player_2_id   
25          h_player_3_id   
26          h_player_4_id   
27          h_player_5_id   
28          h_player_6_id   
29          h_player_7_id   
30          h_player_8_id   
31          h_player_9_id   

                                          

In [49]:
inds = np.arange(player_df.shape[0])
for i in inds:
    if player_df.iloc[i,3] == 'NULL':
        c = '''
        INSERT OR IGNORE INTO person_appearance(
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
            SELECT
                game_id,
                {team_id},
                [{person_id}],
                "{appearance_type_id}"
            FROM game_log        
            WHERE [{person_id}] IS NOT NULL
            
        '''.format(team_id = player_df.iloc[i,1], person_id = player_df.iloc[i,0],
               appearance_type_id = player_df.iloc[i,2] )
    else:
        c = '''
        INSERT OR IGNORE
        INTO person_appearance (
            game_id,
            team_id,
            person_id,
            appearance_type_id)
            SELECT
                game_id,
                {team_id},
                [{person_id}],
                "{appearance_type_id_off}"
            FROM game_log
            WHERE [{person_id}] IS NOT NULL
            
        UNION
            SELECT
                game_id,
                {team_id},
                [{person_id}],
                {appearance_type_id_def}
            FROM game_log
            WHERE [{person_id}] IS NOT NULL
           
        '''.format(team_id = player_df.iloc[i,1], person_id = player_df.iloc[i,0],
                  appearance_type_id_off = player_df.iloc[i,2],
                  appearance_type_id_def = player_df.iloc[i,3])
    run_command(c)
    print(c)
    



        INSERT OR IGNORE INTO person_appearance(
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
            SELECT
                game_id,
                NULL,
                [hp_umpire_id],
                "UHP"
            FROM game_log        
            WHERE [hp_umpire_id] IS NOT NULL
            
        

        INSERT OR IGNORE INTO person_appearance(
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
            SELECT
                game_id,
                NULL,
                [1b_umpire_id],
                "U1B"
            FROM game_log        
            WHERE [1b_umpire_id] IS NOT NULL
            
        

        INSERT OR IGNORE INTO person_appearance(
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
            SELECT
                game_id,
                NULL,
                [2b_umpire_id],
                "U2B"
            FROM game_log     


        INSERT OR IGNORE
        INTO person_appearance (
            game_id,
            team_id,
            person_id,
            appearance_type_id)
            SELECT
                game_id,
                v_name,
                [v_player_6_id],
                "O6"
            FROM game_log
            WHERE [v_player_6_id] IS NOT NULL
            
        UNION
            SELECT
                game_id,
                v_name,
                [v_player_6_id],
                "D" || CAST(v_player_6_def_pos AS INT)
            FROM game_log
            WHERE [v_player_6_id] IS NOT NULL
           
        

        INSERT OR IGNORE
        INTO person_appearance (
            game_id,
            team_id,
            person_id,
            appearance_type_id)
            SELECT
                game_id,
                v_name,
                [v_player_7_id],
                "O7"
            FROM game_log
            WHERE [v_player_7_id] IS NOT NULL
            
        UNI

In [50]:
print(run_query('SELECT COUNT(DISTINCT(game_id)) games FROM game' ))
print(run_query('SELECT COUNT(DISTINCT(game_id)) game_logs FROM game_log' ))
print(run_query('SELECT COUNT(DISTINCT(game_id)) person_games FROM person_appearance'))
print(run_query('SELECT COUNT(DISTINCT(person_id)) person FROM person'))
print(run_query('SELECT COUNT(DISTINCT(person_id)) person_ap FROM person_appearance'))
q = """
SELECT
    *
    FROM person_appearance
    GROUP BY appearance_type_id
"""

run_query(q)

    games
0  171907
   game_logs
0     171907
   person_games
0        171907
   person
0   20494
   person_ap
0      15408


,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1008447,prata101,CL1,18710504FW10,ALP
1,1148676,wrigh101,BS1,18710520BS10,ASP
2,1196694,grimr103,CHN,19210413CHN0,AWB
3,868218,mathb101,FW1,18710504FW10,AWP
4,1646171,mackd101,RC1,18710530BS10,D1
5,1805175,vegaj101,MIN,19800509NYA0,D10
6,1646119,whitd102,CL1,18710504FW10,D2
7,1646129,mcatb101,CH1,18710511CL10,D3
8,1646345,hatfj101,NY2,18710916BS10,D4
9,1646145,watef102,WS3,18710519CH10,D5


In [51]:
q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,318928,porta901,None,20161002WAS0,U1B,First Base,umpire
1,407468,onorb901,None,20161002WAS0,U2B,Second Base,umpire
2,524191,kellj901,None,20161002WAS0,U3B,Third Base,umpire
3,171888,tumpj901,None,20161002WAS0,UHP,Home Plate,umpire
4,1148675,brica001,MIA,20161002WAS0,ALP,Losing Pitcher,award
5,4181195,koeht001,MIA,20161002WAS0,D1,Pitcher,defense
6,2209469,telit001,MIA,20161002WAS0,D2,Catcher,defense
7,3054497,bourj002,MIA,20161002WAS0,D3,1st Base,defense
8,1927793,gordd002,MIA,20161002WAS0,D4,2nd Base,defense
9,2491145,pradm001,MIA,20161002WAS0,D5,3rd Base,defense


In [52]:
run_command('DROP TABLE IF EXISTS game_log')
run_command('DROP TABLE IF EXISTS park_codes')
run_command('DROP TABLE IF EXISTS team_codes')
run_command('DROP TABLE IF EXISTS person_codes')

In [53]:
show_tables()

,name,type
0,person,table
1,park,table
2,appearance_type,table
3,league,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
